# Colab 持久化开发环境工作流

本 Notebook 结合了 **Google Drive** 和 **GitHub**，旨在创建一个稳定、可复现且不会因会话中断而丢失工作的开发环境。

**核心思想:**
1.  **Google Drive**: 作为我们的“持久化硬盘”，存放所有文件，包括代码库、大型数据集和模型文件。
2.  **GitHub**: 作为我们代码的“版本控制系统”，用于追踪和管理所有代码 (`.py`, `.ipynb`) 和配置文件。

## 步骤 1: 环境检查和配置
授权并挂载 Google Drive


运行以下单元格，它会请求授权。授权后，你的 Google Drive 会出现在 `/content/drive/MyDrive/` 目录下，我们的所有工作都将在这里进行。

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 配置Tensor RT开发环境

In [ ]:
# Setup 1 查看系统NVIDIA驱动信息
!nvidia-smi
!nvcc --version

Fri Aug  8 12:53:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

目前(2025-08-10)已知的Colab环境是CUDA 12.4

In [ ]:
# Step 2: Ensure the NVIDIA repository keyring is installed.
echo "INFO: Setting up CUDA repository keyring..."
wget -q https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.1-1_all.deb
sudo dpkg -i cuda-keyring_1.1-1_all.deb
rm cuda-keyring_1.1-1_all.deb
sudo apt-get update

In [ ]:
# Step 3: Install CUDA 12.4 runtime libraries to match the driver.
echo "INFO: Installing CUDA 12.4 runtime libraries..."
sudo apt-get -y install \
    cuda-libraries-12-4 \
    cuda-libraries-dev-12-4

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  cuda-cccl-12-4 cuda-command-line-tools-12-4 cuda-compiler-12-4 cuda-crt-12-4
  cuda-cudart-12-4 cuda-cudart-dev-12-4 cuda-cuobjdump-12-4 cuda-cupti-12-4
  cuda-cupti-dev-12-4 cuda-cuxxfilt-12-4 cuda-documentation-12-4
  cuda-driver-dev-12-4 cuda-gdb-12-4 cuda-libraries-12-4
  cuda-libraries-dev-12-4 cuda-nsight-12-4 cuda-nsight-compute-12-4
  cuda-nsight-systems-12-4 cuda-nvcc-12-4 cuda-nvdisasm-12-4
  cuda-nvml-dev-12-4 cuda-nvprof-12-4 cuda-nvprune-12-4 cuda-nvrtc-12-4
  cuda-nvrtc-dev-12-4 cuda-nvtx-12-4 cuda-nvvm-12-4 cuda-nvvp-12-4
  cuda-opencl-12-4 cuda-opencl-dev-12-4 cuda-profiler-api-12-4
  cuda-sanitizer-12-4 cuda-toolkit-12-4-config-common cuda-tools-12-4
  cuda-visual-tools-12-4 default-jre default-jre-headless fonts-dejavu-core
  fonts-dejavu-extra gds-tools-12-4 libatk-wrapper-java
  libatk-wrapper-java-jni libcublas-12-4

In [ ]:
# Step 4: Forcefully purge any existing TensorRT installations to prevent conflicts.
echo "INFO: Purging any potentially conflicting TensorRT packages..."
sudo apt-get -y purge "libnvinfer*" "tensorrt*" &>/dev/null
sudo apt-get -y autoremove &>/dev/null

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'libnvidia-egl-gbm1' for glob 'libnvidi*'
Note, selecting 'libnvidia-compute-440-server' for glob 'libnvidi*'
Note, selecting 'libnvidia-common-390' for glob 'libnvidi*'
Note, selecting 'libnvidia-common-418' for glob 'libnvidi*'
Note, selecting 'libnvidia-common-430' for glob 'libnvidi*'
Note, selecting 'libnvidia-common-435' for glob 'libnvidi*'
Note, selecting 'libnvidia-common-440' for glob 'libnvidi*'
Note, selecting 'libnvidia-common-450' for glob 'libnvidi*'
Note, selecting 'libnvidia-common-455' for glob 'libnvidi*'
Note, selecting 'libnvidia-common-460' for glob 'libnvidi*'
Note, selecting 'libnvidia-common-465' for glob 'libnvidi*'
Note, selecting 'libnvidia-common-470' for glob 'libnvidi*'
Note, selecting 'libnvidia-common-495' for glob 'libnvidi*'
Note, selecting 'libnvidia-gl-510-server' for glob 'libnvidi*'
Note, selecting 'libnvidia-common-510' for glob 'libnv

In [ ]:
# Step 5: Dynamically and strictly find the correct TensorRT version for CUDA 12.4.
echo "INFO: Searching for a compatible TensorRT version specifically for CUDA 12.4..."
TRT_VERSION_STRING=$(apt-cache madison libnvinfer-dev | grep 'cuda12\.4' | head -n 1 | awk '{print $3}')

if [ -z "$TRT_VERSION_STRING" ]; then
    echo "ERROR: Could not find a compatible TensorRT version for CUDA 12.4 in the apt repositories."
    exit 1
fi

echo "INFO: Found compatible TensorRT version for CUDA 12.4. Pinning installation to ${TRT_VERSION_STRING}"


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libnvinfer-bin libnvinfer-dev libnvinfer-dispatch-dev libnvinfer-dispatch10
  libnvinfer-headers-dev libnvinfer-headers-plugin-dev
  libnvinfer-headers-python-plugin-dev libnvinfer-lean-dev libnvinfer-lean10
  libnvinfer-plugin-dev libnvinfer-plugin10 libnvinfer-samples
  libnvinfer-vc-plugin-dev libnvinfer-vc-plugin10
  libnvinfer-win-builder-resource10 libnvinfer10 libnvonnxparsers-dev
  libnvonnxparsers10 python3-libnvinfer python3-libnvinfer-dev
  python3-libnvinfer-dispatch python3-libnvinfer-lean
The following NEW packages will be installed:
  libnvinfer-bin libnvinfer-dev libnvinfer-dispatch-dev libnvinfer-dispatch10
  libnvinfer-headers-dev libnvinfer-headers-plugin-dev
  libnvinfer-headers-python-plugin-dev libnvinfer-lean-dev libnvinfer-lean10
  libnvinfer-plugin-dev libnvinfer-plugin10 libnvinfer-samples
  libnvinfer-vc-plugi

In [ ]:
# Step 6: Install all necessary TensorRT packages and ALL their dependencies, explicitly pinning every single package version.
# This is the final, most robust solution to prevent apt from choosing incompatible dependency versions.
echo "INFO: Installing TensorRT and all its dependencies with strict version pinning..."
sudo apt-get install -y --allow-downgrades \
    libnvinfer-dev=${TRT_VERSION_STRING} \
    libnvinfer10=${TRT_VERSION_STRING} \
    libnvinfer-headers-dev=${TRT_VERSION_STRING} \
    libnvinfer-headers-plugin-dev=${TRT_VERSION_STRING} \
    libnvonnxparsers-dev=${TRT_VERSION_STRING} \
    libnvonnxparsers10=${TRT_VERSION_STRING} \
    libnvinfer-plugin-dev=${TRT_VERSION_STRING} \
    libnvinfer-plugin10=${TRT_VERSION_STRING} \
    python3-libnvinfer=${TRT_VERSION_STRING} \
    libnvinfer-samples=${TRT_VERSION_STRING} \
    libnvinfer-lean-dev=${TRT_VERSION_STRING} \
    libnvinfer-lean10=${TRT_VERSION_STRING} \
    libnvinfer-dispatch-dev=${TRT_VERSION_STRING} \
    libnvinfer-dispatch10=${TRT_VERSION_STRING} \
    libnvinfer-vc-plugin-dev=${TRT_VERSION_STRING} \
    libnvinfer-vc-plugin10=${TRT_VERSION_STRING}

echo "INFO: Successfully installed CUDA 12.4 libraries and version-pinned TensorRT."


In [ ]:
# Step 7: Verify the installation.
echo "INFO: Verifying installations..."
echo "NVIDIA Driver (from nvidia-smi):"
nvidia-smi | grep "CUDA Version"
echo "NVCC Version (may still show a different version, but runtime is now aligned):"
nvcc --version
echo "TensorRT (dpkg) Version:"
dpkg -l | grep libnvinfer

echo "INFO: Environment setup complete. The runtime environment is now aligned with the CUDA 12.4 driver."

安装Python的开发库

In [ ]:
!pip install onnx onnxruntime

In [ ]:
# 1. 验证Python API
import tensorrt
print(f"TensorRT version: {tensorrt.__version__}")
tensorrt.Builder(tensorrt.Logger())
# assert tensorrt.Builder(tensorrt.Logger())

# 2. 验证 trtexec 命令行工具
# pip安装后，trtexec通常位于 /usr/local/bin/
# !which trtexec
# !trtexec --help

# print("\n✅ TensorRT environment setup complete!" )

## 步骤 2: 配置 GitHub (首次运行需要)

为了让 Colab 能够访问你的 GitHub 仓库，你需要进行两项配置：

1.  **创建 GitHub Personal Access Token (PAT)**
    - 前往 GitHub **Settings -> Developer settings -> Personal access tokens -> Tokens (classic)**.
    - 点击 **Generate new token**，勾选 `repo` 权限，生成一个 Token 并**立即复制保存好**。

2.  **在 Colab 中保存 Token**
    - 在 Colab 左侧边栏找到并点击 **密钥 (🔑)** 图标。
    - 点击 **“添加新密钥”**。
    - 名称填写 `GITHUB_TOKEN`。
    - 值粘贴你刚才生成的 PAT。
    - **确保“允许笔记本访问”开关是打开的。**

## 步骤 3: 克隆或同步你的代码仓库

此单元格会自动执行以下操作：
- 定义在 Google Drive 中的项目路径。


In [ ]:
!git clone your_repo_url

## 步骤 4: 日常开发工作

**在shell命令里, 工作目录切换到了你的代码库中 (`repo_path`)**，可以直接运行脚本或执行命令,修改代码等。

In [ ]:
!pip install -r requirements.txt